# 03 - Statistical Analysis & Hypothesis Testing

Rigorous statistical analysis of Stadium vs Federation/Club projects.

**Research Hypotheses:**
- **H1**: Stadium projects exhibit significantly lower contributor entropy than Federation projects
- **H2**: Stadium projects show higher Gini coefficients (contribution inequality)
- **H3**: Stadium entropy correlates with VSM S2 (coordination) metrics
- **H4**: Stadium projects have faster PR merge times (centralized decision-making)
- **H5**: Stadium projects have fewer governance files
- **H6**: Entropy predicts project classification with >80% accuracy

## Setup

In [ ]:
import json
import sys
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import mannwhitneyu, ttest_ind, pearsonr, spearmanr
import statsmodels.api as sm
from statsmodels.stats.power import TTestIndPower
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix

# Add src to path
sys.path.insert(0, '../src')
from analysis.entropy_calculation import EntropyCalculator

# Set plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = [12, 6]

print("✅ Setup complete!")

## 1. Load and Prepare Data

In [ ]:
# Load all collected project data
data_dir = Path("../data/raw")
data_files = list(data_dir.glob("*_data.json"))

print(f"Found {len(data_files)} data file(s)")

projects = []
entropy_calc = EntropyCalculator()

for file_path in data_files:
    with open(file_path, 'r') as f:
        data = json.load(f)
    
    repo = data['repository']
    contributors = data.get('contributors', [])
    maintainers = data.get('maintainers', {}).get('statistics', {})
    pr_stats = data.get('pull_requests', {}).get('statistics', {})
    issue_stats = data.get('issues', {}).get('statistics', {})
    gov_files = data.get('governance_files', {})
    
    # Calculate entropy metrics
    if contributors:
        entropy, normalized_entropy = entropy_calc.contributor_entropy(contributors)
        contributions = [c.get('contributions', 0) for c in contributors]
        gini = entropy_calc.gini_coefficient(contributions)
        
        total_contrib = sum(contributions)
        top_pct = contributions[0] / total_contrib * 100 if total_contrib > 0 else 0
        top_2_pct = sum(contributions[:2]) / total_contrib * 100 if total_contrib > 0 else 0
    else:
        entropy, normalized_entropy, gini = 0, 0, 0
        top_pct, top_2_pct = 0, 0
    
    # Count governance files
    gov_count = sum(1 for v in gov_files.values() if v)
    
    projects.append({
        'repository': repo.get('full_name', 'Unknown'),
        'language': repo.get('language', 'Unknown'),
        'stars': repo.get('stargazers_count', 0),
        'forks': repo.get('forks_count', 0),
        'contributors': len(contributors),
        'active_maintainers': maintainers.get('active_maintainers_6mo', 0),
        'commits': len(data.get('recent_commits', [])),
        
        # Entropy metrics
        'entropy': entropy,
        'normalized_entropy': normalized_entropy,
        'gini': gini,
        'top_contributor_pct': top_pct,
        'top_2_pct': top_2_pct,
        
        # PR metrics (VSM S2 - Coordination)
        'total_prs': pr_stats.get('total_prs', 0),
        'merge_rate': pr_stats.get('merged_count', 0) / max(pr_stats.get('total_prs', 1), 1),
        'avg_merge_time': pr_stats.get('avg_time_to_merge', 0),
        'conflict_rate': pr_stats.get('conflict_rate', 0),
        
        # Issue metrics (VSM S1 - Operations)
        'total_issues': issue_stats.get('total_issues', 0),
        'avg_close_time': issue_stats.get('avg_time_to_close', 0),
        
        # Governance
        'governance_files': gov_count,
    })

df = pd.DataFrame(projects)
print(f"\nLoaded {len(df)} projects")
df.head()

## 2. Classify Projects

Classify based on entropy and dominance metrics.

In [ ]:
def classify_project(row):
    """Classify project as Stadium, Federation, or Club."""
    
    # Stadium indicators
    low_maintainers = row['active_maintainers'] <= 3
    low_entropy = row['normalized_entropy'] < 0.5
    high_dominance = row['top_contributor_pct'] > 40
    high_gini = row['gini'] > 0.7
    
    # Federation indicators  
    many_maintainers = row['active_maintainers'] > 5
    high_entropy = row['normalized_entropy'] > 0.6
    distributed = row['top_contributor_pct'] < 25
    
    # Score-based classification
    stadium_score = sum([low_maintainers, low_entropy, high_dominance, high_gini])
    federation_score = sum([many_maintainers, high_entropy, distributed])
    
    if stadium_score >= 3:
        return 'Stadium'
    elif stadium_score >= 2 and federation_score < 2:
        return 'Stadium'
    elif federation_score >= 2:
        return 'Federation'
    else:
        return 'Club'  # Hybrid/uncertain

df['classification'] = df.apply(classify_project, axis=1)

print("Project Classifications:")
print(df['classification'].value_counts())
print("\n" + "="*60)
df[['repository', 'active_maintainers', 'normalized_entropy', 'top_contributor_pct', 'gini', 'classification']]

## 3. Power Analysis

Determine if we have sufficient sample size for meaningful results.

In [ ]:
# Power analysis for two-sample t-test
power_analysis = TTestIndPower()

# Expected effect size (Cohen's d)
effect_sizes = [0.5, 0.8, 1.0, 1.2]  # medium to large
alpha = 0.05
power = 0.80

print("Required Sample Sizes (per group) for 80% Power:")
print("="*50)
for d in effect_sizes:
    n = power_analysis.solve_power(effect_size=d, alpha=alpha, power=power, alternative='two-sided')
    print(f"  Cohen's d = {d}: n = {int(np.ceil(n))} per group")

# Calculate achieved power with current sample
n_stadium = len(df[df['classification'] == 'Stadium'])
n_other = len(df[df['classification'] != 'Stadium'])
n_min = min(n_stadium, n_other)

print(f"\nCurrent Sample:")
print(f"  Stadium: {n_stadium}")
print(f"  Other: {n_other}")

if n_min > 2:
    for d in effect_sizes:
        achieved_power = power_analysis.solve_power(effect_size=d, nobs1=n_min, alpha=alpha, alternative='two-sided')
        print(f"\nAchieved power for d={d}: {achieved_power:.2%}")

## 4. Hypothesis Testing

### H1: Stadium projects have lower contributor entropy

In [ ]:
# Split groups
stadium = df[df['classification'] == 'Stadium']['normalized_entropy']
non_stadium = df[df['classification'] != 'Stadium']['normalized_entropy']

print("H1: Stadium projects have lower contributor entropy")
print("="*60)

if len(stadium) >= 2 and len(non_stadium) >= 2:
    # Descriptive statistics
    print(f"\nDescriptive Statistics:")
    print(f"  Stadium (n={len(stadium)}):     mean={stadium.mean():.3f}, std={stadium.std():.3f}")
    print(f"  Non-Stadium (n={len(non_stadium)}): mean={non_stadium.mean():.3f}, std={non_stadium.std():.3f}")
    
    # Effect size (Cohen's d)
    pooled_std = np.sqrt(((len(stadium)-1)*stadium.std()**2 + (len(non_stadium)-1)*non_stadium.std()**2) / 
                         (len(stadium) + len(non_stadium) - 2))
    cohens_d = (non_stadium.mean() - stadium.mean()) / pooled_std if pooled_std > 0 else 0
    print(f"\n  Cohen's d: {cohens_d:.3f} ({'large' if abs(cohens_d) > 0.8 else 'medium' if abs(cohens_d) > 0.5 else 'small'})")
    
    # Mann-Whitney U test (non-parametric)
    stat, p_value = mannwhitneyu(stadium, non_stadium, alternative='less')
    print(f"\nMann-Whitney U Test:")
    print(f"  U-statistic: {stat:.2f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Result: {'✅ SIGNIFICANT' if p_value < 0.05 else '❌ Not significant'} (α=0.05)")
    
    # t-test (parametric)
    t_stat, t_p = ttest_ind(stadium, non_stadium, alternative='less')
    print(f"\nt-test (for reference):")
    print(f"  t-statistic: {t_stat:.2f}")
    print(f"  p-value: {t_p:.4f}")
else:
    print("\n⚠️  Insufficient data for statistical testing")
    print(f"    Stadium: {len(stadium)}, Non-Stadium: {len(non_stadium)}")

### H2: Stadium projects have higher Gini coefficients

In [ ]:
stadium_gini = df[df['classification'] == 'Stadium']['gini']
non_stadium_gini = df[df['classification'] != 'Stadium']['gini']

print("H2: Stadium projects have higher Gini coefficients")
print("="*60)

if len(stadium_gini) >= 2 and len(non_stadium_gini) >= 2:
    print(f"\nDescriptive Statistics:")
    print(f"  Stadium (n={len(stadium_gini)}):     mean={stadium_gini.mean():.3f}, std={stadium_gini.std():.3f}")
    print(f"  Non-Stadium (n={len(non_stadium_gini)}): mean={non_stadium_gini.mean():.3f}, std={non_stadium_gini.std():.3f}")
    
    stat, p_value = mannwhitneyu(stadium_gini, non_stadium_gini, alternative='greater')
    print(f"\nMann-Whitney U Test:")
    print(f"  U-statistic: {stat:.2f}")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Result: {'✅ SIGNIFICANT' if p_value < 0.05 else '❌ Not significant'} (α=0.05)")
else:
    print("\n⚠️  Insufficient data for statistical testing")

### H3: Entropy correlates with VSM S2 (Coordination) metrics

In [ ]:
print("H3: Entropy correlates with coordination metrics")
print("="*60)

# Correlation between entropy and conflict rate (VSM S2 indicator)
valid_data = df[df['conflict_rate'] > 0]

if len(valid_data) >= 3:
    # Pearson correlation
    r, p = pearsonr(valid_data['normalized_entropy'], valid_data['conflict_rate'])
    print(f"\nEntropy vs Conflict Rate:")
    print(f"  Pearson r: {r:.3f}")
    print(f"  p-value: {p:.4f}")
    print(f"  Result: {'✅ SIGNIFICANT' if p < 0.05 else '❌ Not significant'}")
    
    # Spearman correlation (non-parametric)
    rho, p_spearman = spearmanr(valid_data['normalized_entropy'], valid_data['conflict_rate'])
    print(f"\n  Spearman ρ: {rho:.3f} (p={p_spearman:.4f})")
else:
    print("\n⚠️  Insufficient data with conflict rates")

# Correlation with merge time
valid_merge = df[df['avg_merge_time'] > 0]
if len(valid_merge) >= 3:
    r, p = pearsonr(valid_merge['normalized_entropy'], valid_merge['avg_merge_time'])
    print(f"\nEntropy vs Merge Time:")
    print(f"  Pearson r: {r:.3f}")
    print(f"  p-value: {p:.4f}")

### H4: Stadium projects have faster PR merge times

In [ ]:
stadium_merge = df[(df['classification'] == 'Stadium') & (df['avg_merge_time'] > 0)]['avg_merge_time']
non_stadium_merge = df[(df['classification'] != 'Stadium') & (df['avg_merge_time'] > 0)]['avg_merge_time']

print("H4: Stadium projects have faster PR merge times")
print("="*60)

if len(stadium_merge) >= 2 and len(non_stadium_merge) >= 2:
    print(f"\nDescriptive Statistics (hours):")
    print(f"  Stadium (n={len(stadium_merge)}):     median={stadium_merge.median():.1f}, mean={stadium_merge.mean():.1f}")
    print(f"  Non-Stadium (n={len(non_stadium_merge)}): median={non_stadium_merge.median():.1f}, mean={non_stadium_merge.mean():.1f}")
    
    stat, p_value = mannwhitneyu(stadium_merge, non_stadium_merge, alternative='less')
    print(f"\nMann-Whitney U Test:")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Result: {'✅ SIGNIFICANT' if p_value < 0.05 else '❌ Not significant'}")
else:
    print("\n⚠️  Insufficient data for statistical testing")

### H5: Stadium projects have fewer governance files

In [ ]:
stadium_gov = df[df['classification'] == 'Stadium']['governance_files']
non_stadium_gov = df[df['classification'] != 'Stadium']['governance_files']

print("H5: Stadium projects have fewer governance files")
print("="*60)

if len(stadium_gov) >= 2 and len(non_stadium_gov) >= 2:
    print(f"\nDescriptive Statistics:")
    print(f"  Stadium (n={len(stadium_gov)}):     mean={stadium_gov.mean():.2f}, median={stadium_gov.median():.0f}")
    print(f"  Non-Stadium (n={len(non_stadium_gov)}): mean={non_stadium_gov.mean():.2f}, median={non_stadium_gov.median():.0f}")
    
    stat, p_value = mannwhitneyu(stadium_gov, non_stadium_gov, alternative='less')
    print(f"\nMann-Whitney U Test:")
    print(f"  p-value: {p_value:.4f}")
    print(f"  Result: {'✅ SIGNIFICANT' if p_value < 0.05 else '❌ Not significant'}")
else:
    print("\n⚠️  Insufficient data for statistical testing")

### H6: Entropy predicts classification with >80% accuracy

In [ ]:
print("H6: Entropy predicts classification with >80% accuracy")
print("="*60)

# Prepare features and target
feature_cols = ['normalized_entropy', 'gini', 'top_contributor_pct']
X = df[feature_cols].values
y = (df['classification'] == 'Stadium').astype(int).values

if len(df) >= 10 and y.sum() >= 2 and (len(y) - y.sum()) >= 2:
    # Logistic regression with cross-validation
    model = LogisticRegression(random_state=42, max_iter=1000)
    
    # Cross-validation
    cv_scores = cross_val_score(model, X, y, cv=min(5, len(df)//2), scoring='accuracy')
    
    print(f"\nLogistic Regression (features: {feature_cols})")
    print(f"  Cross-validation accuracy: {cv_scores.mean():.1%} (±{cv_scores.std():.1%})")
    print(f"  Result: {'✅ >80% ACHIEVED' if cv_scores.mean() > 0.80 else '❌ <80%'}")
    
    # Fit on all data for coefficients
    model.fit(X, y)
    print(f"\nFeature Importance (coefficients):")
    for feat, coef in zip(feature_cols, model.coef_[0]):
        print(f"  {feat}: {coef:.3f}")
else:
    print("\n⚠️  Insufficient data for classification model")
    print(f"    Total samples: {len(df)}, Stadium: {y.sum()}, Other: {len(y) - y.sum()}")

## 5. Visualization

In [ ]:
if len(df) >= 2:
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # 1. Entropy distribution by classification
    ax1 = axes[0, 0]
    for cls in df['classification'].unique():
        data = df[df['classification'] == cls]['normalized_entropy']
        ax1.hist(data, alpha=0.5, label=f"{cls} (n={len(data)})", bins=10)
    ax1.set_xlabel('Normalized Entropy')
    ax1.set_ylabel('Count')
    ax1.set_title('Entropy Distribution by Classification')
    ax1.legend()
    
    # 2. Entropy vs Gini scatter
    ax2 = axes[0, 1]
    for cls in df['classification'].unique():
        subset = df[df['classification'] == cls]
        ax2.scatter(subset['normalized_entropy'], subset['gini'], 
                   label=cls, s=100, alpha=0.7)
    ax2.set_xlabel('Normalized Entropy')
    ax2.set_ylabel('Gini Coefficient')
    ax2.set_title('Entropy vs Gini by Classification')
    ax2.legend()
    
    # 3. Top contributor dominance
    ax3 = axes[1, 0]
    df_sorted = df.sort_values('top_contributor_pct', ascending=True)
    colors = ['coral' if c == 'Stadium' else 'steelblue' for c in df_sorted['classification']]
    ax3.barh(range(len(df_sorted)), df_sorted['top_contributor_pct'], color=colors)
    ax3.set_yticks(range(len(df_sorted)))
    ax3.set_yticklabels([r[:20] for r in df_sorted['repository']])
    ax3.set_xlabel('Top Contributor %')
    ax3.set_title('Top Contributor Dominance (orange=Stadium)')
    ax3.axvline(x=40, color='red', linestyle='--', alpha=0.5, label='40% threshold')
    
    # 4. Summary metrics comparison
    ax4 = axes[1, 1]
    metrics = ['normalized_entropy', 'gini', 'active_maintainers']
    x = np.arange(len(metrics))
    width = 0.35
    
    stadium_means = df[df['classification'] == 'Stadium'][metrics].mean()
    other_means = df[df['classification'] != 'Stadium'][metrics].mean()
    
    # Normalize for comparison
    max_vals = df[metrics].max()
    stadium_norm = stadium_means / max_vals
    other_norm = other_means / max_vals
    
    ax4.bar(x - width/2, stadium_norm, width, label='Stadium', color='coral')
    ax4.bar(x + width/2, other_norm, width, label='Other', color='steelblue')
    ax4.set_xticks(x)
    ax4.set_xticklabels(['Entropy', 'Gini', 'Maintainers'])
    ax4.set_ylabel('Normalized Value')
    ax4.set_title('Key Metrics Comparison')
    ax4.legend()
    
    plt.tight_layout()
    plt.savefig('../data/processed/statistical_analysis.png', dpi=150, bbox_inches='tight')
    plt.show()
    print("\n📊 Figure saved to data/processed/statistical_analysis.png")
else:
    print("⚠️  Need more data for visualizations")

## 6. Summary Report

In [ ]:
print("\n" + "="*70)
print("STATISTICAL ANALYSIS SUMMARY REPORT")
print("="*70)

print(f"\nDataset:")
print(f"  Total projects: {len(df)}")
for cls in df['classification'].value_counts().items():
    print(f"  {cls[0]}: {cls[1]}")

print(f"\nKey Findings:")
print(f"  Stadium normalized entropy: {df[df['classification']=='Stadium']['normalized_entropy'].mean():.3f} (mean)")
print(f"  Non-Stadium normalized entropy: {df[df['classification']!='Stadium']['normalized_entropy'].mean():.3f} (mean)")
print(f"  Stadium Gini coefficient: {df[df['classification']=='Stadium']['gini'].mean():.3f} (mean)")
print(f"  Stadium top contributor %: {df[df['classification']=='Stadium']['top_contributor_pct'].mean():.1f}% (mean)")

print(f"\nHypothesis Results:")
print(f"  H1 (Lower entropy): {'Needs more data' if len(stadium) < 2 else 'See above'}")
print(f"  H2 (Higher Gini): {'Needs more data' if len(stadium) < 2 else 'See above'}")
print(f"  H3 (Correlation): {'Needs more data' if len(valid_data) < 3 else 'See above'}")
print(f"  H4 (Faster merge): {'Needs more data' if len(stadium_merge) < 2 else 'See above'}")
print(f"  H5 (Fewer gov files): {'Needs more data' if len(stadium_gov) < 2 else 'See above'}")
print(f"  H6 (>80% accuracy): {'Needs more data' if len(df) < 10 else 'See above'}")

print(f"\nRecommendations:")
if len(df) < 30:
    print(f"  ⚠️  Collect more Stadium projects (target: 28-30)")
    print(f"     Current: {len(df[df['classification']=='Stadium'])} Stadium projects")
if len(df[df['classification'] != 'Stadium']) < 15:
    print(f"  ⚠️  Need Federation/Club control projects for comparison")

print("\n" + "="*70)

In [ ]:
# Export results
output_dir = Path("../data/processed")
output_dir.mkdir(parents=True, exist_ok=True)

# Save analysis DataFrame
df.to_csv(output_dir / "statistical_analysis_results.csv", index=False)
print(f"\n✅ Results saved to {output_dir / 'statistical_analysis_results.csv'}")